# TiTiler-CMR: Sea Surface Temperature Example

The MUR SST dataset has daily records for sea surface temperature and ice cover fraction. There is a netcdf file for each record.

To run the titiler-cmr service locally you can fire up the docker network with this command:
```bash
docker compose up
```

#### Requirements
To run some of the chunks in this notebook you will need to install a few packages:
- `earthaccess`
- `folium`
- `httpx`
- `xarray`

`!pip install folium httpx earthaccess xarray`

In [ ]:
import json
from datetime import datetime, timezone

import earthaccess
import httpx
import xarray as xr
from folium import GeoJson, Map, TileLayer

# if running titiler-cmr in the docker network
titiler_endpoint = "http://localhost:8081"

## Identify the dataset

You can find the MUR SST dataset using the `earthaccess.search_datasets` function.

In [ ]:
datasets = earthaccess.search_datasets(doi="10.5067/GHGMR-4FJ04")
ds = datasets[0]

concept_id = ds["meta"]["concept-id"]
print("Concept-Id: ", concept_id)

print("Abstract: ", ds["umm"]["Abstract"])

## Examine a granule

Each granule contains a single day record for the entire globe and has a single data file.  

In [ ]:
results = earthaccess.search_data(
    count=1,
    concept_id=concept_id,
    temporal=("2024-10-12", "2024-10-13"),
)
print("Granules:")
print(results)
print()
print("Example of NetCDF URL: ")
for link in results[0].data_links(access="external"):
    print(link)

## Explore the available variables

The NetCDF file can be opened with xarray using the `h5netcdf` engine. When running outside of AWS region us-west-2 you will need to access the data using "external" `https` links (rather than "direct" `s3` links). Those links will require authentication which is handled by `earthaccess` as long as you have your Earthdata credentials stored in the `~/.netrc` file!

In [ ]:
fs = earthaccess.get_fsspec_https_session()

ds = xr.open_dataset(fs.open(results[0].data_links(access="external")[0]), engine="h5netcdf")
print("Data Variables:")
for var in ds.data_vars:
    print(str(var))

ds

## Define a query for titiler-cmr

To use titiler-cmr's endpoints for a NetCDF dataset like this we need to define a date range for the CMR query and a `variable` to analyze.

In [ ]:
variable = "sea_ice_fraction"
datetime_range = "/".join(
    dt.isoformat() for dt in [datetime(2024, 10, 10, tzinfo=timezone.utc), datetime(2024, 10, 11, tzinfo=timezone.utc)]
)

## Display tiles in an interactive map

The `/tilejson.json` endpoint will provide a parameterized `xyz` tile URL that can be added to an interactive map.

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/WebMercatorQuad/tilejson.json",
    params = (
        ("concept_id", concept_id),
        # Datetime in form of `start_date/end_date`
        ("datetime", datetime_range),
        # titiler-cmr can work with both Zarr and COG dataset
        # but we need to tell the endpoints in advance which backend
        # to use
        ("backend", "xarray"),
        ("variable", variable),
        # We need to set min/max zoom because we don't want to use lowerzoom level (e.g 0)
        # which will results in useless large scale query
        ("minzoom", 2),
        ("maxzoom", 13),
        ("rescale", "0,1"),
        ("colormap_name", "blues_r"),
    )
).json()

print(r)

In [ ]:
bounds = r["bounds"]
m = Map(
    location=(80, -40),
    zoom_start=3
)

TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="NASA",
).add_to(m)
m

## GeoJSON Statistics
The `/statistics` endpoint can be used to get summary statistics for a geojson `Feature` or `FeatureCollection`.

In [ ]:
geojson_dict = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              -20.79973248834736,
              83.55979308678764
            ],
            [
              -20.79973248834736,
              75.0115425216471
            ],
            [
              14.483337068956956,
              75.0115425216471
            ],
            [
              14.483337068956956,
              83.55979308678764
            ],
            [
              -20.79973248834736,
              83.55979308678764
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

r = httpx.post(
    f"{titiler_endpoint}/statistics",
    params=(
        ("concept_id", concept_id),
        # Datetime in form of `start_date/end_date`
        ("datetime", datetime_range),
        # titiler-cmr can work with both Zarr and COG dataset
        # but we need to tell the endpoints in advance which backend
        # to use
        ("backend", "xarray"),
        ("variable", variable),
    ),
    json=geojson_dict,
    timeout=60,
).json()

print(json.dumps(r, indent=2))